In [ ]:
library(Seurat)
library(dplyr)

# reference data set

ref_data = readRDS('/tmp/work/Visium/zenodo.11468564/scRNA_data/panB_scRNA_processed_data.rds')
my36colors <-c('#E5D2DD', '#53A85F', '#F1BB72', '#F3B1A0', '#D6E7A3', '#57C3F3', '#476D87',
               '#E95C59', '#E59CC4', '#AB3282', '#23452F', '#BD956A', '#8C549C', '#585658',
               '#9FA3A8', '#E0D4CA', '#5F3D69', '#C5DEBA', '#58A4C3', '#E4C755', '#F7F398',
               '#AA9A59', '#E63863', '#E39A35', '#C1E6F3', '#6778AE', '#91D0BE', '#B53E2B',
               '#712820', '#DCC1DD', '#CCE0F5',  '#CCC9E6', '#625D9E', '#68A180', '#3A6963',
               '#968175'
)

#figure 11b
DimPlot(ref_data,group.by = "celltype",label=T, cols = my36colors) + NoLegend()

#SCtransform on reference data
ref_data <- SCTransform(ref_data, ncells = 3000, verbose = FALSE) %>%
    RunPCA(verbose = FALSE) %>%
    RunUMAP(dims = 1:30)

saveRDS('b_cell_ref_data.rds')

#after SCTransform
DimPlot(ref_data,group.by = "celltype",label=T, cols = my36colors) + NoLegend()

#visium data
visium_samples <- readRDS('/tmp/work/Visium/BRCA_2024/objects/seurat_objects.rds')

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`



In [1]:
visium_samples <- readRDS('/tmp/work/Visium/BRCA_2024/objects/seurat_objects.rds')
visium_samples

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: Seurat



An object of class Seurat 
36140 features across 32625 samples within 2 assays 
Active assay: SCT (18055 features, 3000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: Spatial
 8 spatial fields of view present: slice1 slice1.2 slice1.3 slice1.4 slice1.5 slice1.6 slice1.7 slice1.8

In [3]:
anchors <- FindTransferAnchors(reference = ref_data, query = visium_samples, normalization.method = "SCT")
predictions.assay <- TransferData(anchorset = anchors, refdata = ref_data$celltype, prediction.assay = TRUE,
    weight.reduction = visium_samples[["pca"]], dims = 1:30)
visium_samples[["predictions"]] <- predictions.assay

An object of class Seurat 
27557 features across 474718 samples within 1 assay 
Active assay: RNA (27557 features, 1819 variable features)
 3 layers present: counts, data, scale.data
 3 dimensional reductions calculated: pca, harmony, umap

In [ ]:
library(Seurat)
library(future)
setwd("/tmp/work/Visium/BRCA_2024/objects")
options(future.globals.maxSize = 50 * 1024^3)  #50 GiB
plan("multicore", workers = 20)
ref_data = readRDS('/tmp/work/Visium/zenodo.11468564/scRNA_data/panB_scRNA_processed_data.rds')
ref_data <- SCTransform(ref_data, ncells = 3000, verbose = FALSE) %>%
    RunPCA(verbose = FALSE) %>%
    RunUMAP(dims = 1:30)
saveRDS('b_cell_ref_data.rds')
